In [315]:
import pandas as pd
import numpy as np
import glob
import cv2 as cv

In [494]:
BASE_PATH = r"C:\Users\abdul\Desktop\CS 512 - Computer vision\Homework\HW4\data"
GOOD_TEST_DATA_PATH= BASE_PATH+ r"\non_planar_test_data.csv"
NOISY_DATA_PATH_1 = BASE_PATH+ r"\noisy_test_data_1.csv"
NOISY_DATA_PATH_2 = BASE_PATH+ r"\noisy_test_data_2.csv"
IMAGES_PATH = BASE_PATH+ r"\images\*.jpg"
OUTPUT_PATH = BASE_PATH+ r"\out"
POINTS_FROM_SAVED_IMAGE = BASE_PATH+ r"\out\chess10_img_pts.csv"

In [495]:
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
obj_point = np.zeros((6*7,3), np.float32)
obj_point[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)


In [485]:
# Arrays to store object points and image points from all the images.
# 3d point in real world space
obj_points = []
 # 2d points in image plane.
img_points = []

images = glob.glob(IMAGES_PATH)
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (7,6), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        print('img: ', fname)
        obj_points.append(obj_point*20)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        img_points.append(corners2)
        
        # Draw and display the corners
        cv.drawChessboardCorners(img, (7,6), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)
        
cv.destroyAllWindows()

img:  C:\Users\abdul\Desktop\CS 512 - Computer vision\Homework\HW4\data\images\left01.jpg
img:  C:\Users\abdul\Desktop\CS 512 - Computer vision\Homework\HW4\data\images\left03.jpg
img:  C:\Users\abdul\Desktop\CS 512 - Computer vision\Homework\HW4\data\images\left04.jpg
img:  C:\Users\abdul\Desktop\CS 512 - Computer vision\Homework\HW4\data\images\left05.jpg
img:  C:\Users\abdul\Desktop\CS 512 - Computer vision\Homework\HW4\data\images\left06.jpg
img:  C:\Users\abdul\Desktop\CS 512 - Computer vision\Homework\HW4\data\images\left07.jpg
img:  C:\Users\abdul\Desktop\CS 512 - Computer vision\Homework\HW4\data\images\left08.jpg
img:  C:\Users\abdul\Desktop\CS 512 - Computer vision\Homework\HW4\data\images\left12.jpg
img:  C:\Users\abdul\Desktop\CS 512 - Computer vision\Homework\HW4\data\images\left13.jpg
img:  C:\Users\abdul\Desktop\CS 512 - Computer vision\Homework\HW4\data\images\left14.jpg


In [486]:
def save_img_points(img_points, obj_points):
    '''
    input: img_points -- a list of shape [n_images,n_points,1,2]
            obj_points -- a list of shape [n_images,n_points,3]
    '''
    img_points = np.array(img_points)
    obj_points = np.array(obj_points)
    shape = img_points.shape
    img_points = img_points.reshape([shape[0],shape[1],shape[-1]])
    for i in range(img_points.shape[0]):
        df = pd.DataFrame(img_points[i], columns=['x_img','y_img'])
        df2 = pd.DataFrame(obj_points[i], columns=['x_w','y_w','z_w'])
        df3 = pd.concat([df2,df], axis=1)
        df3.to_csv(OUTPUT_PATH + r"\chess{}_img_pts.csv".format(i+1),
                  encoding='utf-8', index=False)

In [487]:
# save the image points found as csv
save_img_points(img_points, obj_points)

In [488]:
def evaluate_fit(points_df, M):
    total_error = 0
    for i in range(points_df.shape[0]):
        world_point = list(points_df.loc[i, ['x_w','y_w','z_w']])
        world_point.extend([1])
        x_img = points_df.x_img.values[i]
        y_img = points_df.y_img.values[i]
        temp = np.linalg.norm([x_img-(M[0].dot(world_point) / M[2].dot(world_point))],ord=2) ** 2 + \
                np.linalg.norm([y_img-(M[1].dot(world_point) / M[2].dot(world_point))],ord=2) ** 2
        total_error += temp
    total_error /= points_df.shape[0]
    total_error = np.sqrt(total_error)
    
    return total_error

In [493]:
def calibration(point_file):
    '''
    input: point_file -- a csv file containing world points(x_w,y_w,z_w) and image
            points(x_img,y_img) as column names
    '''
    points_df = pd.read_csv(point_file)
    
    A_matrix = []
#     create A_matrix
    for i in range(points_df.shape[0]):
        world_point = list(points_df.loc[i, ['x_w','y_w','z_w']])
        world_point.extend([1])
        r1 = world_point.copy()
        r1.extend([0.0,0.0,0.0,0.0])
        x_img = points_df.x_img.values[i]
        temp = [-x_img*i for i in world_point]
        r1.extend(temp)
        A_matrix.append(r1)

        r3 = [0.0,0.0,0.0,0.0]
        r3.extend(world_point)
        y_img = points_df.y_img.values[i]
        temp2 = [-y_img*i for i in world_point]
        r3.extend(temp2)
        A_matrix.append(r3)
    A_matrix = np.array(A_matrix)
        
    zero_matrix = np.zeros((A_matrix.shape[0], 1))
    
#     solve Ax=0 with SVD
    U,s,Vt = np.linalg.svd(A_matrix)
    smallest = np.argmin(s)
    M = Vt[smallest]
    M = np.reshape(M, (3,4))
    
#     solve for intrinsic camera parameters
    a_matx = M[:,:3]
    a3 = a_matx[-1]; a1 = a_matx[0]; a2 = a_matx[1]
    a3_norm = np.linalg.norm(a3)
    scale_mag = 1/a3_norm
    u_knot = scale_mag**2 * a1.dot(a3)
    v_knot = scale_mag**2 * a2.dot(a3)
    alpha_v = np.sqrt(scale_mag**2 * a2.dot(a2) - v_knot**2)
    alpha_u = np.sqrt(scale_mag**2 * a1.dot(a1) - skew**2 - u_knot**2)
    K = np.array([[alpha_u, skew, u_knot],[0, alpha_v, v_knot],[0, 0, 1]])
    
#     solve for extrinsic camera parameters
    b = M[:, -1]
    epsilon = np.sign(b[-1])
    T = epsilon*scale_mag*(np.matmul(np.linalg.inv(K), b))
    
    r1 = scale_mag**2 / (alpha_v) *np.cross(a2,a3)
    r3 = epsilon*scale_mag*a3
    r2 = np.cross(r3,r1)
    R = np.array([r1,r2,r3])
    
#   evaluate fitting error
    total_error = evaluate_fit(points_df, M)
        
    return (K, T, R, total_error)

In [499]:
K, T, R, error = calibration(GOOD_TEST_DATA_PATH)

In [500]:
def show_results(K, T, R, error):
    print('Intrinsic camera parameters:\n----------------------------------\n')
    print('(alpha_u, alpha_v): ', (K[0,0],K[1,1]) )
    print('skew: ', K[0,1])
    print('(Uo,Vo): ', (K[0,2],K[1,2]))
    
    print('\nExtrinsic camera parameters:\n----------------------------------\n')
    print('Rotation: ', R)
    print('Translation: ', T)
    
    print('Error in fit: ', error)

In [501]:
show_results(K, T, R, error)

Intrinsic camera parameters:
----------------------------------

(alpha_u, alpha_v):  (491.3318693106586, 495.93049552038354)
skew:  0.0
(Uo,Vo):  (312.01561915892444, 212.28232543256848)

Extrinsic camera parameters:
----------------------------------

Rotation:  [[-0.77214303  0.63524585 -0.01605778]
 [ 0.41336662  0.48293491 -0.77194683]
 [-0.48262116 -0.60269111 -0.63548426]]
Translation:  [ 13.25883875  44.83081557 845.73240224]
Error in fit:  7.273199491291778
